# 코로나 전후 라이프스타일 게임 플레이 양상 분석

#### 사용한 데이터 셋
* [Steam Monthly Player Data](https://www.kaggle.com/connorwynkoop/steam-monthly-player-data)
* [코로나 19 발생 현황](https://kosis.kr/statHtml/statHtml.do?mode=noLogo&orgId=101&tblId=DT_COVID19_004)

In [1]:
import pandas as pd
import numpy as np
from datetime import datetime

In [2]:
# sd is SteamData
sd_df = pd.read_csv("AllSteamData.csv", encoding="utf-8-sig")
display(sd_df.head(5))

,Name,Month,Avg. Players,Gain,% Gain,Peak Players
0,Counter-Strike,Last 30 Days,8488.74,27.6,0.33%,15065
1,Counter-Strike,Sep-21,8461.12,-390.07,-4.41%,14559
2,Counter-Strike,Aug-21,8851.19,-286.57,-3.14%,14064
3,Counter-Strike,Jul-21,9137.76,-359.69,-3.79%,14972
4,Counter-Strike,Jun-21,9497.45,-1243.5,-11.58%,16391


In [3]:
# 게임 목록 조회
# gl is GameList
gl_df = sd_df.groupby("Name").count().reset_index()[["Name", "Month"]]
gl_df.columns = ["Name", "Number of Data"]
display(gl_df.shape)
display(gl_df)

(38164, 2)

,Name,Number of Data
0,! That Bastard Is Trying To Steal Our Gold !,53
1,"!""Time Lock VR-1""!",37
2,"!""We Are The Dwarves""!",69
3,!4RC4N01D! 2: Retro Edition,20
4,!4RC4N01D! 3: Cold Space 5000 Achievements!,20
...,...,...
38159,이 AI를 복구해주세요.,3
38160,이라,2
38161,잭과 콩나무: 숨겨진 이야기,12
38162,🌈,1


### 최소 충분 개월수 계산
위의 목록 중 데이터 표본이 적어 분석에 부적합한(코로나 전/후로 나눌 수 없는) 게임 제거

In [4]:
first_outbreak = datetime(2019, 12, 1)
now = datetime.now()
num_month = (now.year - first_outbreak.year) * 12 + (now.month - first_outbreak.month)
min_month = num_month * 2 # 현재와 코로나 19 최초 발견 사이의 개월수의 두배에 달하는 개월수를 적정 표본 수로 규정
                          # 즉, 해당 숫자보다 적은 행을 갖는 게임은 코로나 전후에 따른 추이를 분석하기에 불충분한 데이터로 판단

print(f"최소 충분 개월수: {min_month}")

gl_df = gl_df[gl_df["Number of Data"] >= min_month].reset_index(drop=True) # 최소 충분 개월수를 만족하는 게임 목록
display(gl_df)
gl_df.to_csv("gamelist.csv", index=False, encoding="utf-8-sig")

최소 충분 개월수: 46


,Name,Number of Data
0,! That Bastard Is Trying To Steal Our Gold !,53
1,"!""We Are The Dwarves""!",69
2,"""Glow Ball"" - The billiard puzzle game",51
3,#killallzombies,67
4,#monstercakes,55
...,...,...
7705,東方憑依華 ～ Antinomy of Common Flowers.,47
7706,永遠消失的幻想鄉 ～ The Disappearing of Gensokyo,46
7707,神明的一天世界-God's One Day World,48
7708,神舞幻想 Faith of Danschant,47


### 데이터 전처리

In [5]:
# 전처리-1 : 불필요한 행(Last 30 Days) 제거
sd_df.drop(sd_df[sd_df["Month"] == "Last 30 Days"].index, inplace=True)

In [6]:
# 전처리-2 : 개월수가 충족되지 않은 불충분 데이터 제거
sd_df = sd_df[sd_df["Name"].isin(list(gl_df["Name"]))]

In [7]:
# 전처리-3 : Month 열의 값 형태 변경 ('Sep-21' -> '2021-10')
sd_df["Month"] = sd_df[["Month"]].apply(
    lambda x: datetime.strptime(x["Month"], "%b-%y").strftime("%Y-%m"), axis=1
)

In [8]:
# 전처리-4 : 미사용 열 제거(Gain, % Gain, Peak Plaers)
sd_df.drop(["Gain", "% Gain", "Peak Players"], axis=1, inplace=True)

In [9]:
# 전처리-5 : Avg. Players 열 정규화
# mp is Max Players
mp_df = sd_df.groupby("Name").max().reset_index()[["Name", "Avg. Players"]]
mp_df.columns = ["Name", "Max Players"]

# Max Player 행 추가
sd_df = pd.merge(sd_df, mp_df, on="Name", how="inner")

# 최대 플레이어 수를 100으로 뒀을 때의 수치를 계산하여 Avg. Players 열 정규화
sd_df["Avg. Players"] = sd_df[["Avg. Players", "Max Players"]].apply(
    lambda x: x["Avg. Players"] / x["Max Players"] * 100 if x["Max Players"] != 0 else np.nan, axis=1
)

# Max Player 행 제거(미사용)
sd_df.drop(["Max Players"], axis=1, inplace=True)

display(sd_df)

,Name,Month,Avg. Players
0,Counter-Strike,2021-09,24.303458
1,Counter-Strike,2021-08,25.423883
2,Counter-Strike,2021-07,26.247017
3,Counter-Strike,2021-06,27.280180
4,Counter-Strike,2021-05,30.851970
...,...,...,...
569069,Conqueror's Blade,2019-06,76.570748
569070,Conqueror's Blade,2019-05,1.679975
569071,Conqueror's Blade,2019-04,2.679419
569072,Conqueror's Blade,2019-03,0.221169


### 생활 시뮬레이션 게임 분류
gamelist.csv의 목록에서 약식으로 임의 선정

In [10]:
sim_life = ["VRChat", "Terraria", "The Sims(TM) 3", "The Sims(TM) Medieval", "Don't Starve", "Don't Starve Together", "Don't Starve Mod Tools", "Stardew Valley"]

# Simulation Life
sl_df = sd_df[sd_df["Name"].isin(list(sim_life)) == True]
display(sl_df)

,Name,Month,Avg. Players
105297,The Sims(TM) 3,2021-09,33.541931
105298,The Sims(TM) 3,2021-08,36.316152
105299,The Sims(TM) 3,2021-07,34.992746
105300,The Sims(TM) 3,2021-06,36.077563
105301,The Sims(TM) 3,2021-05,38.062592
...,...,...,...
447421,VRChat,2017-05,0.129269
447422,VRChat,2017-04,0.136118
447423,VRChat,2017-03,0.162045
447424,VRChat,2017-02,0.282753


In [11]:
# Not Simulation Life
nsl_df = sd_df[sd_df["Name"].isin(list(sim_life)) == False]
display(nsl_df)

,Name,Month,Avg. Players
0,Counter-Strike,2021-09,24.303458
1,Counter-Strike,2021-08,25.423883
2,Counter-Strike,2021-07,26.247017
3,Counter-Strike,2021-06,27.280180
4,Counter-Strike,2021-05,30.851970
...,...,...,...
569069,Conqueror's Blade,2019-06,76.570748
569070,Conqueror's Blade,2019-05,1.679975
569071,Conqueror's Blade,2019-04,2.679419
569072,Conqueror's Blade,2019-03,0.221169


Month 열을 기준으로 평균 Avg. Players 값 산출

In [12]:
sl_mean_df = sl_df.groupby("Month").mean()
sl_mean_df.reset_index()

,Month,Avg. Players
0,2012-07,36.151272
1,2012-08,22.277351
2,2012-09,17.186643
3,2012-10,14.647293
4,2012-11,17.568967
...,...,...
106,2021-05,49.483311
107,2021-06,56.192439
108,2021-07,50.361262
109,2021-08,53.277539


In [13]:
nsl_mean_df = nsl_df.groupby("Month").mean()
nsl_mean_df.reset_index()

,Month,Avg. Players
0,2012-07,51.068003
1,2012-08,36.973945
2,2012-09,31.458780
3,2012-10,28.943006
4,2012-11,30.329966
...,...,...
106,2021-05,24.925632
107,2021-06,24.836513
108,2021-07,26.361746
109,2021-08,25.116995


# 코로나 추이
두 데이터 셋의 원본 데이터의 행의 형태가 일치하지 않으므로, 전처리를 거친 후 병합해야함

In [14]:
covid2020_df = pd.read_csv("101_DT_COVID19_004_20211001150023.csv", encoding="euc-kr")
covid2021_df = pd.read_csv("101_DT_COVID19_004_20211001150440.csv", encoding="euc-kr")

display(covid2020_df)
display(covid2021_df)

,국가별,항목,단위,2020. 05. 01,2020. 05. 02,2020. 05. 03,2020. 05. 04,2020. 05. 05,2020. 05. 06,2020. 05. 07,...,2020. 12. 22,2020. 12. 23,2020. 12. 24,2020. 12. 25,2020. 12. 26,2020. 12. 27,2020. 12. 28,2020. 12. 29,2020. 12. 30,2020. 12. 31
0,한국,확진자수,명,10774.0,10780.0,10793.0,10801.0,10793.0,10806.0,10810.0,...,51460.0,52550,53533,54770,55902.0,56872.0,57680.0,58725.0,59773,60740.0
1,한국,사망자수,명,248.0,250.0,250.0,252.0,250.0,255.0,256.0,...,722.0,739,756,773,793.0,808.0,819.0,859.0,879,900.0
2,아프가니스탄,확진자수,명,2171.0,2171.0,2469.0,2469.0,2469.0,3224.0,3392.0,...,50678.0,50888,51070,51357,51595.0,51764.0,51848.0,52007.0,52147,52330.0
3,아프가니스탄,사망자수,명,64.0,64.0,72.0,72.0,72.0,95.0,104.0,...,2074.0,2089,2105,2126,2135.0,2148.0,2158.0,2170.0,2182,2189.0
4,바레인,확진자수,명,3037.0,3169.0,3243.0,3356.0,3464.0,3679.0,3842.0,...,90282.0,90450,90634,90817,91070.0,91304.0,91518.0,91733.0,91935,92169.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
373,솔로몬제도,사망자수,명,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,0,0,0,NaN,NaN,NaN,NaN,0,NaN
374,바누아투,확진자수,명,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,1,1,1,NaN,NaN,NaN,NaN,1,NaN
375,바누아투,사망자수,명,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,0,0,0,NaN,NaN,NaN,NaN,0,NaN
376,기타지역 영토,확진자수,명,5773.0,5991.0,6036.0,6284.0,6573.0,6666.0,6819.0,...,NaN,294612,297686,300748,NaN,NaN,NaN,NaN,313861,NaN


,국가별,항목,단위,2021. 01. 01,2021. 01. 02,2021. 01. 03,2021. 01. 04,2021. 01. 05,2021. 01. 06,2021. 01. 07,...,2021. 09. 21,2021. 09. 22,2021. 09. 23,2021. 09. 24,2021. 09. 25,2021. 09. 28,2021. 09. 29,2021. 09. 30,2021. 10. 01,Unnamed: 221
0,한국,확진자수,명,61769.0,62593.0,63244.0,64264.0,64979.0,65818.0,66686.0,...,289263.0,290983.0,292699.0,295132.0,298402.0,305842.0,308725.0,311286.0,313773.0,NaN
1,한국,사망자수,명,917.0,942.0,962.0,981.0,1007.0,1027.0,1046.0,...,2413.0,2419.0,2427.0,2434.0,2441.0,2464.0,2474.0,2481.0,2497.0,NaN
2,아프가니스탄,확진자수,명,52330.0,52513.0,52586.0,52709.0,52909.0,53011.0,53105.0,...,154585.0,154712.0,154757.0,154800.0,154960.0,155072.0,155093.0,155128.0,155174.0,NaN
3,아프가니스탄,사망자수,명,2189.0,2201.0,2211.0,2221.0,2230.0,2237.0,2244.0,...,7199.0,7199.0,7199.0,7199.0,7199.0,7200.0,7201.0,7204.0,7204.0,NaN
4,바레인,확진자수,명,92425.0,92675.0,92913.0,NaN,93478.0,93766.0,93995.0,...,274264.0,274383.0,274452.0,274524.0,274524.0,274814.0,274877.0,274925.0,274981.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
377,솔로몬제도,사망자수,명,0.0,0.0,NaN,NaN,0.0,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN
378,바누아투,확진자수,명,1.0,1.0,NaN,NaN,1.0,NaN,NaN,...,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,NaN
379,바누아투,사망자수,명,0.0,0.0,NaN,NaN,0.0,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN
380,기타지역 영토,확진자수,명,319585.0,321938.0,NaN,NaN,328058.0,NaN,NaN,...,941587.0,946639.0,950342.0,953733.0,956554.0,965894.0,969168.0,973105.0,975797.0,NaN


In [15]:
# 국가를 고유 키 값으로 만들기 위해 사망자수 행 제거
covid2020_df.drop(covid2020_df[covid2020_df["항목"] == "사망자수"].index, inplace=True)
covid2021_df.drop(covid2021_df[covid2021_df["항목"] == "사망자수"].index, inplace=True)

# 항목 & 단위 열 제거
covid2020_df.drop(["항목", "단위"], axis=1, inplace=True)
covid2021_df.drop(["항목", "단위"], axis=1, inplace=True)

# 병합
covid_df = pd.merge(covid2020_df, covid2021_df, on="국가별", how="inner")
display(covid_df)

,국가별,2020. 05. 01,2020. 05. 02,2020. 05. 03,2020. 05. 04,2020. 05. 05,2020. 05. 06,2020. 05. 07,2020. 05. 08,2020. 05. 09,...,2021. 09. 21,2021. 09. 22,2021. 09. 23,2021. 09. 24,2021. 09. 25,2021. 09. 28,2021. 09. 29,2021. 09. 30,2021. 10. 01,Unnamed: 221
0,한국,10774.0,10780.0,10793.0,10801.0,10793.0,10806.0,10810.0,10822.0,10840.0,...,289263.0,290983.0,292699.0,295132.0,298402.0,305842.0,308725.0,311286.0,313773.0,NaN
1,아프가니스탄,2171.0,2171.0,2469.0,2469.0,2469.0,3224.0,3392.0,3563.0,3563.0,...,154585.0,154712.0,154757.0,154800.0,154960.0,155072.0,155093.0,155128.0,155174.0,NaN
2,바레인,3037.0,3169.0,3243.0,3356.0,3464.0,3679.0,3842.0,4131.0,4404.0,...,274264.0,274383.0,274452.0,274524.0,274524.0,274814.0,274877.0,274925.0,274981.0,NaN
3,방글라데시,7103.0,7667.0,8238.0,8790.0,9455.0,10143.0,10929.0,11719.0,12425.0,...,1544238.0,1545800.0,1547176.0,1548320.0,1549553.0,1552563.0,1552563.0,1553873.0,1555911.0,NaN
4,부탄,7.0,7.0,7.0,7.0,7.0,7.0,7.0,7.0,7.0,...,2597.0,2599.0,2599.0,2599.0,2599.0,2599.0,2599.0,2600.0,2601.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
184,뉴질랜드,1129.0,1132.0,1134.0,1136.0,1136.0,1137.0,1138.0,1139.0,1141.0,...,3725.0,3739.0,3763.0,3779.0,3788.0,3838.0,3848.0,3892.0,3917.0,NaN
185,파푸아뉴기니,8.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0,...,18933.0,18968.0,19069.0,19182.0,19278.0,19523.0,19703.0,19703.0,20221.0,NaN
186,솔로몬제도,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,20.0,20.0,20.0,20.0,20.0,20.0,20.0,20.0,20.0,NaN
187,바누아투,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,NaN


In [16]:
# 행·열 간 전환
covid_df = covid_df.transpose()

# 국가 이름을 행으로 설정
covid_df = covid_df.rename(columns=covid_df.iloc[0])
covid_df.drop(covid_df.index[0], inplace=True)
display(covid_df)

,한국,아프가니스탄,바레인,방글라데시,부탄,브루나이,캄보디아,중국,동티모르,홍콩,...,에스와티니,남수단,호주,피지,마셜제도,뉴질랜드,파푸아뉴기니,솔로몬제도,바누아투,기타지역 영토
2020. 05. 01,10774.0,2171.0,3037.0,7103.0,7.0,138.0,122.0,82874.0,24.0,1037.0,...,91.0,34.0,6746.0,18.0,NaN,1129.0,8.0,NaN,NaN,5773.0
2020. 05. 02,10780.0,2171.0,3169.0,7667.0,7.0,138.0,122.0,82875.0,24.0,1039.0,...,100.0,35.0,6762.0,18.0,NaN,1132.0,8.0,NaN,NaN,5991.0
2020. 05. 03,10793.0,2469.0,3243.0,8238.0,7.0,138.0,122.0,82877.0,24.0,1039.0,...,106.0,45.0,6767.0,18.0,NaN,1134.0,8.0,NaN,NaN,6036.0
2020. 05. 04,10801.0,2469.0,3356.0,8790.0,7.0,138.0,122.0,82880.0,24.0,1039.0,...,108.0,45.0,6783.0,18.0,NaN,1136.0,8.0,NaN,NaN,6284.0
2020. 05. 05,10793.0,2469.0,3464.0,9455.0,7.0,138.0,122.0,82881.0,24.0,1040.0,...,112.0,46.0,6801.0,18.0,NaN,1136.0,8.0,NaN,NaN,6573.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021. 09. 28,305842.0,155072.0,274814.0,1552563.0,2599.0,6540.0,109087.0,96081.0,19398.0,12196.0,...,45770.0,11948.0,97540.0,50755.0,4.0,3838.0,19523.0,20.0,3.0,965894.0
2021. 09. 29,308725.0,155093.0,274877.0,1552563.0,2599.0,6700.0,109926.0,96106.0,19445.0,12209.0,...,45770.0,11948.0,99032.0,50807.0,4.0,3848.0,19703.0,20.0,3.0,969168.0
2021. 09. 30,311286.0,155128.0,274925.0,1553873.0,2600.0,6813.0,110792.0,96128.0,19445.0,12214.0,...,45870.0,11981.0,100912.0,50872.0,4.0,3892.0,19703.0,20.0,3.0,973105.0
2021. 10. 01,313773.0,155174.0,274981.0,1555911.0,2601.0,6950.0,111673.0,96162.0,19498.0,12217.0,...,45904.0,11995.0,102723.0,50953.0,4.0,3917.0,20221.0,20.0,3.0,975797.0


In [17]:
# 결측치 제거(Unnamed: 21)
covid_df.drop(covid_df.index[-1], inplace=True)
display(covid_df)

,한국,아프가니스탄,바레인,방글라데시,부탄,브루나이,캄보디아,중국,동티모르,홍콩,...,에스와티니,남수단,호주,피지,마셜제도,뉴질랜드,파푸아뉴기니,솔로몬제도,바누아투,기타지역 영토
2020. 05. 01,10774.0,2171.0,3037.0,7103.0,7.0,138.0,122.0,82874.0,24.0,1037.0,...,91.0,34.0,6746.0,18.0,NaN,1129.0,8.0,NaN,NaN,5773.0
2020. 05. 02,10780.0,2171.0,3169.0,7667.0,7.0,138.0,122.0,82875.0,24.0,1039.0,...,100.0,35.0,6762.0,18.0,NaN,1132.0,8.0,NaN,NaN,5991.0
2020. 05. 03,10793.0,2469.0,3243.0,8238.0,7.0,138.0,122.0,82877.0,24.0,1039.0,...,106.0,45.0,6767.0,18.0,NaN,1134.0,8.0,NaN,NaN,6036.0
2020. 05. 04,10801.0,2469.0,3356.0,8790.0,7.0,138.0,122.0,82880.0,24.0,1039.0,...,108.0,45.0,6783.0,18.0,NaN,1136.0,8.0,NaN,NaN,6284.0
2020. 05. 05,10793.0,2469.0,3464.0,9455.0,7.0,138.0,122.0,82881.0,24.0,1040.0,...,112.0,46.0,6801.0,18.0,NaN,1136.0,8.0,NaN,NaN,6573.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021. 09. 25,298402.0,154960.0,274524.0,1549553.0,2599.0,5697.0,106619.0,95986.0,19308.0,12176.0,...,45633.0,11899.0,92179.0,50447.0,4.0,3788.0,19278.0,20.0,3.0,956554.0
2021. 09. 28,305842.0,155072.0,274814.0,1552563.0,2599.0,6540.0,109087.0,96081.0,19398.0,12196.0,...,45770.0,11948.0,97540.0,50755.0,4.0,3838.0,19523.0,20.0,3.0,965894.0
2021. 09. 29,308725.0,155093.0,274877.0,1552563.0,2599.0,6700.0,109926.0,96106.0,19445.0,12209.0,...,45770.0,11948.0,99032.0,50807.0,4.0,3848.0,19703.0,20.0,3.0,969168.0
2021. 09. 30,311286.0,155128.0,274925.0,1553873.0,2600.0,6813.0,110792.0,96128.0,19445.0,12214.0,...,45870.0,11981.0,100912.0,50872.0,4.0,3892.0,19703.0,20.0,3.0,973105.0


In [18]:
# 일별 확진자수 합계로 열 통합
covid_df = covid_df.fillna(0)
covid_df["Total"] = covid_df.sum(1)
covid_df = covid_df[["Total"]]
display(covid_df)

,Total
2020. 05. 01,3204353.0
2020. 05. 02,3292996.0
2020. 05. 03,3374678.0
2020. 05. 04,3455718.0
2020. 05. 05,3523684.0
...,...
2021. 09. 25,230419936.0
2021. 09. 28,231703606.0
2021. 09. 29,232076416.0
2021. 09. 30,232637324.0


In [19]:
# 월별 확진자수 평균으로 행 통합
covid_df["Month"] = covid_df.index
covid_df["Month"] = covid_df[["Month"]].apply(
    lambda x: "-".join(x["Month"].split(". ")[0:2]), axis=1
)

covid_df = covid_df.groupby("Month").mean().reset_index()
display(covid_df)

,Month,Total
0,2020-05,4.519882e+06
1,2020-06,7.997920e+06
2,2020-07,1.356829e+07
3,2020-08,2.127057e+07
4,2020-09,2.611423e+07
5,2020-10,2.644152e+07
6,2020-11,2.844507e+07
7,2020-12,4.526535e+07
8,2021-01,5.756747e+07
9,2021-02,4.033034e+07
